In [6]:
pip install librosa 


   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ---------------------------------------  256.0/260.1 kB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 260.1/260.1 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ----------------------- ---------------- 0.6/1.0 MB 19.2 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.0 MB 16.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/164.9 kB ? eta -:--:--
   ---------------------------------------- 164.9/164.9 kB 9.7 MB/s eta 0:00:00


In [12]:
pip install sounddevice

   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   --- ------------------------------------ 30.7/363.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------  358.4/363.6 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 363.6/363.6 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [33]:
import librosa
import numpy as np
import os

In [35]:
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

In [37]:
# Example of loading and processing a dataset
def load_dataset(dataset_path):
    features = []
    labels = []
    for emotion in os.listdir(dataset_path):
        for file in os.listdir(os.path.join(dataset_path, emotion)):
            if file.endswith('.wav'):
                file_path = os.path.join(dataset_path, emotion, file)
                mfccs = extract_features(file_path)
                features.append(mfccs)
                labels.append(emotion)
    return np.array(features), np.array(labels)

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [41]:
# Load dataset
features, labels = load_dataset('Voice_Emotion_Detection_Dataset')

In [42]:
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

In [43]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

In [44]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(labels_encoded)), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

C:\Users\Mihir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.2531 - loss: 16.3762 - val_accuracy: 0.4393 - val_loss: 1.6247
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6080 - loss: 1.1346 - val_accuracy: 0.7393 - val_loss: 0.7366
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7970 - loss: 0.5792 - val_accuracy: 0.7893 - val_loss: 0.5645
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8808 - loss: 0.3987 - val_accuracy: 0.8321 - val_loss: 0.4899
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8849 - loss: 0.3650 - val_accuracy: 0.8393 - val_loss: 0.4787
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8859 - loss: 0.3742 - val_accuracy: 0.8821 - val_loss: 0.3646
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9179 - loss: 0.2646 - val_accuracy: 0.8750 - val_loss: 0.3640
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9248 - loss: 0.2425 - val_accuracy: 0.8679 - val_los

In [45]:
pip install wavio

Note: you may need to restart the kernel to use updated packages.


In [46]:
import tkinter as tk
from tkinter import filedialog, messagebox
import sounddevice as sd
import wavio

In [47]:
class EmotionDetectorApp:
    def __init__(self, master):
        self.master = master
        master.title("Emotion Detection through Voice")

        self.label = tk.Label(master, text="Upload a voice note or record your voice:")
        self.label.pack()

        self.upload_button = tk.Button(master, text="Upload Voice Note", command=self.upload_file)
        self.upload_button.pack()

        self.record_button = tk.Button(master, text="Record Voice", command=self.record_voice)
        self.record_button.pack()

    def upload_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.process_audio(file_path)

    def record_voice(self):
        fs = 44100  # Sample rate
        seconds = 5  # Duration of recording
        recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
        sd.wait()  # Wait until recording is finished
        wavio.write("recording.wav", recording, fs, sampwidth=2)
        self.process_audio("recording.wav")

    def process_audio(self, file_path):
        # Check if the voice is female (this is a placeholder, implement your own logic)
        if not self.is_female_voice(file_path):
            messagebox.showerror(" Error", "Please upload a female voice.")
            return
        # Extract features and make predictions
        features = extract_features(file_path)
        prediction = model.predict(np.array([features]))
        emotion = le.inverse_transform([np.argmax(prediction)])
        messagebox.showinfo("Detected Emotion", f"The detected emotion is: {emotion[0]}")

    def is_female_voice(self, file_path):
        # Placeholder for actual voice gender detection logic
        return True  # Assume all uploaded voices are female for now


In [48]:
root = tk.Tk()
app = EmotionDetectorApp(root)
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [49]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes, target_names=le.classes_))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
                    precision    recall  f1-score   support

             Angry       0.91      0.86      0.88        35
           Disgust       0.89      1.00      0.94        42
              Fear       0.88      0.90      0.89        42
             Happy       0.98      0.98      0.98        43
           Neutral       0.92      0.96      0.94        46
Pleasant_Surprised       0.97      0.90      0.94        40
               Sad       1.00      0.91      0.95        32

          accuracy                           0.93       280
         macro avg       0.94      0.93      0.93       280
      weighted avg       0.93      0.93      0.93       280

